# Import things

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import random
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
random.seed(42)

# Load data

In [2]:
red_wine = pd.read_csv("./resources/resources/winequality-red.csv", sep=';')

# V.1 Exploring the green reds

In [3]:
red_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
red_wine.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

## a)

In [5]:
def plot_scatter_matrix(wine_data, good_threshhold, bad_threshold, save_plot=False):
    """Visualizes the data
    """
    length = len(wine_data.columns)
    fig, ax = plt.subplots(length, length, figsize=(50, 50))
    bigger = wine_data.where(wine_data['quality'] > good_threshhold)
    smaller = wine_data.where(wine_data['quality'] < bad_threshold)
    
    for i in range(length):
        for j in range(length):
            if i == j:
                ax[(i, j)].text(0.5, 0.5, '\n'.join(wine_data.columns[i].split()), fontsize=35, ha='center', va='center')
            else:
                ax[(j, i)].plot(bigger.iloc[:, i], bigger.iloc[:, j], 'co')
                ax[(j, i)].plot(smaller.iloc[:, i], smaller.iloc[:, j], 'mo')
    if save_plot:
        plt.savefig('scatter_matrix.png')
# plot_scatter_matrix(red_wine, 6, 5)

In [52]:
# plot_scatter_matrix(red_wine, 7, 4)

## b)

I think it can be pH, sulphates, alcohol and total sulfur dioxide. Cause there
different
groups of wine(based on quality) r better splitted.

# V.2 Learning to perceptron

In [7]:
data_3_8 = red_wine[(red_wine['quality'] >= 8) | (red_wine['quality'] <= 3)].reset_index(drop=True)
y_3_8 = [1 if i == True else 0 for i in data_3_8['quality'] >= 8]

In [8]:
def display(ret, dot=100, detail=10000, final=False):
    """Displays text info about training process"""
    if final:
        print()
        print('DONE')
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
        print('final weights = ', ret[-1][2], ', bias = ', ret[-1][3])
    if ret[-1][0] == 0:
        return
    if ret[-1][0] % dot == 0:
        print('.', end='')
    if ret[-1][0] % detail == 0:
        print()
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
        print('weights change = ', ret[-1][2][0] - ret[-detail - 1][2][0], ret[-1][2][1] - ret[-detail - 1][2][1])
        print('weights = ', ret[-1][2], 'bias = ', ret[-1][3])
        print('bias change = ', ret[-1][3] - ret[-detail - 1][3])

## a), b)

In [9]:
class perceptron:
    """A simple perceptron, with randomly initialized weights and bias"""

    def __init__(self, amount):
        """Random data initialization"""
        random.seed(42)
        self.amount = amount
        self.weights = [random.random() for n in range(amount)]
        self.bias = random.random()
        for i in range(amount):
            print('Weights[', i, '] = ', self.weights[i], sep='')
        print('bias = ', self.bias, '\n')

    def activation(self, value):
        """Activation function"""
        return 1 if value > 0 else 0

    def predict(self, data, alpha = None, y = None):
        """Predicts answer and updates weights to fit the data item better
        Returns predicted value or whether he guessed right
        """
        rez = self.activation(sum([data[i] * self.weights[i]
                                   for i in range(self.amount)]) + self.bias)
        if y == None:
            return rez
        if rez != y:
            self.weights = [self.weights[i] + alpha * data[i] * (y - rez) 
                            for i in range(self.amount)]
            self.bias = self.bias + alpha * (y - rez)
        return rez != y

    def train(self, data, y, epochs = 0, alpha = 0.001):
        """Trains perceptron epoch times, and logs results"""
        ret = []
        errors = 1
        if epochs == 0:
            i = 0
            while (errors):
                errors = 0
                for item in data.itertuples():
                    errors += self.predict(item[1:], alpha, y[item[0]])
                ret.append([i, errors, self.weights, self.bias])
                i += 1
                display(ret)
            display(ret, final=True)
        else:
            for i in range(epochs):
                errors = 0
                for item in data.itertuples():
                    errors += self.predict(item[1:], alpha, y[item[0]])
                ret.append([i, errors, self.weights, self.bias])
                display(ret)
                if errors == 0:
                    break
            display(ret, final=True)
        return ret

In [10]:
model = perceptron(2)
logs = model.train(data_3_8[['pH', 'alcohol']], y_3_8, 0, 0.01)

Weights[0] = 0.6394267984578837
Weights[1] = 0.025010755222666936
bias =  0.27502931836911926 

....................................................................................................
cycle = 10000, errors = 2
weights change =  -7.069099999996053 1.848000000000045
weights =  [-6.536073201538169, 1.653510755222712] bias =  4.455029318369069
bias change =  4.209999999999949
...............................
DONE
cycle = 13154, errors = 0
final weights =  [-7.259373201536793, 1.731510755222712] , bias =  6.135029318369033


## c)

In [11]:
def plot_performance(performance, wine_data, good_threshold, bad_threshold, epoch=-1, save_plot=False, debug=False):
    """Plot the performance of my perceptron"""
    if debug:
        fig, ax = plt.subplots(1, 2, figsize=(30, 30))
    else:
        fig, ax = plt.subplots(1, 2)#, figsize=(10, 10))
    bigger = wine_data.where(wine_data['quality'] > good_threshold).reset_index(drop=True)
    smaller = wine_data.where(wine_data['quality'] < bad_threshold).reset_index(drop=True)
    
    ax[0].plot([i[1] for i in performance], 'g')
    ax[0].set_title('Errors as a function of epoch')
    ax[0].set_ylabel('Classification errors')
    ax[0].set_xlabel('Epoch')
    ax[1].plot(bigger['alcohol'], bigger['pH'], 'mo', label = 'good wines (> ' + str(good_threshold) + ' score)')
    ax[1].plot(smaller['alcohol'], smaller['pH'], 'co', label = 'bad wines (< ' + str(bad_threshold) + ' score)')
    ax[1].set_title('Decision boundary on epoch:' + str(len(performance) if epoch == -1 else epoch))
    ax[1].set_xlabel('Alcohol')
    ax[1].set_ylabel('pH')
    
    val = {}
    if debug:
        val['alcohol'] = [-100, 100]         
    else:
#         val['alcohol'] = [5, 15]         
        val['alcohol'] = [min(wine_data['alcohol']) - 1, max(wine_data['alcohol']) + 1] 
    val['pH'] = [
            - (performance[epoch][2][1] * val['alcohol'][0] + performance[epoch][3]) / performance[epoch][2][0],
            - (performance[epoch][2][1] * val['alcohol'][1] + performance[epoch][3]) / performance[epoch][2][0]
        ]
    ax[1].plot(val['alcohol'], val['pH'], 'g--')
    if debug:
        ax[1].set_xlim(-100, 100)
        ax[1].set_ylim(-100, 100)
        ax[1].fill_between(val['alcohol'], val['pH'], -100, facecolor='#f8bbd0')
        ax[1].fill_between(val['alcohol'], val['pH'], 100, facecolor='#b2ebf2')
    else:
#         ax[1].set_xlim(5, 15)
#         ax[1].set_ylim(2, 5)
        ax[1].fill_between(val['alcohol'], val['pH'], -5, facecolor='#f8bbd0')
        ax[1].fill_between(val['alcohol'], val['pH'], 15, facecolor='#b2ebf2')
        ax[1].set_xlim(min(wine_data['alcohol']) - 1, max(wine_data['alcohol']) + 1)
        ax[1].set_ylim(min(wine_data['pH']) - 1, max(wine_data['pH']) + 1)
#         ax[1].fill_between(val['alcohol'], val['pH'], min(wine_data['pH']), facecolor='#b2ebf2')
#         ax[1].fill_between(val['alcohol'], val['pH'], max(wine_data['pH']), facecolor='#f8bbd0')
    plt.tight_layout()
    plt.legend(loc='upper center', bbox_to_anchor=(1.6, 1))
    plt.show()

In [12]:
interactive(plot_performance, performance=fixed(logs), wine_data=fixed(red_wine), good_threshold=(0, 10),\
            bad_threshold=(0, 10), epoch=(0, len(logs) - 1))

interactive(children=(IntSlider(value=5, description='good_threshold', max=10), IntSlider(value=5, description…

## d) Feature scaling

In [13]:
normalized_red_wine = red_wine.copy()
for title in red_wine.columns:
    if not title == 'quality':
        normalized_red_wine[title] = (red_wine[title] - min(red_wine[title])) / (max(red_wine[title]) - min(red_wine[title]))
normalized_data_3_8 = normalized_red_wine[(red_wine['quality'] >= 8) | (red_wine['quality'] <= 3)].reset_index(drop=True)

In [14]:
model_normalized = perceptron(2)
logs_normalized = model_normalized.train(normalized_data_3_8[['pH', 'alcohol']], y_3_8, 0, 0.01)

Weights[0] = 0.6394267984578837
Weights[1] = 0.025010755222666936
bias =  0.27502931836911926 


DONE
cycle = 85, errors = 0
final weights =  [-0.09521887083345507, 0.10070306291497405] , bias =  0.015029318369119145


In [15]:
interactive(plot_performance, performance=fixed(logs_normalized), wine_data=fixed(normalized_red_wine), good_threshold=(0, 10),\
            bad_threshold=(0, 10), epoch=(0, len(logs_normalized) - 1))

interactive(children=(IntSlider(value=5, description='good_threshold', max=10), IntSlider(value=5, description…

# V.3 My fair ADALINE

## a)

Using stochastic learning algorithm instead of batch. Not adding polynomial features and not using more complicated algorhithms. No parallelism. Using only labels in learning, not values.

## b), c)

In [65]:
class adaline:
    """A simple adaline, with randomly initialized weights and bias"""

    def __init__(self, amount):
        """Random data initialization"""
        self.amount = amount
        self.weights = [random.random() for n in range(amount)]
        self.bias = random.random()

        
    def quantizer(self, value):
        """Quantizer function"""
        return 1 if value > 6.35 else 0

    
    def predict(self, data, y=False):
        """Predicts answer and updates weights to fit the data item better
        Returns predicted value or whether he guessed right
        """
        rez = sum([data[i] * self.weights[i] for i in range(self.amount)]) + self.bias
        if y == False:
            return self.quantizer(rez)
        else:
            err = y - rez
#             print('y =', y, 'rez =', rez, 'y - rez =', y - rez)
            return [[x * err for x in [*data, 1]], err]
        
    def train(self, data, y, epochs = 0, alpha = 0.001):
        """Trains perceptron epoch times, and logs results"""
        ret = []
        if epochs == 0:
            i = 0
            while (True):
                error = 0
                delta = [0] * (self.amount + 1)
                for item in data.itertuples():
                    [rez, err] = self.predict(item[1:], y[item[0]])
                    delta = [delta[i] + rez[i] for i in range(self.amount + 1)]
                    error += err
                self.weights = [self.weights[i] + alpha * delta[i] / len(data) for i in range(self.amount)]
                self.bias += alpha * delta[-1] / len(data)
                ret.append([i, error, self.weights, self.bias - 5])
                display(ret)
                if i > 0 and abs(ret[-1][1] - error) < 0.001:
                    break
                i += 1
            display(ret, final=True)
        else:
            for i in range(epochs):
                error = 0
                delta = [0] * (self.amount + 1)
                for item in data.itertuples():
                    [rez, err] = self.predict(item[1:], y[item[0]])
                    delta = [delta[i] + rez[i] for i in range(self.amount + 1)]
                    error += err ** 2
#                 print(delta, errors)
                self.weights = [self.weights[i] + alpha * delta[i] / len(data) for i in range(self.amount)]
                self.bias += alpha * delta[-1] / len(data)
                ret.append([i, error, self.weights, self.bias - 5])
                display(ret)
                if i > 0 and abs(ret[-2][1] - error) < 0.0000000001:
                    print(ret[-1], error)
                    break
            display(ret, final=True)
        return ret

In [17]:
data_4_7 = red_wine[(red_wine['quality'] >= 7) | (red_wine['quality'] <= 4)].reset_index(drop=True)
y_4_7 = data_4_7['quality']

In [54]:
model = adaline(2)
adaline_logs = model.train(data_4_7[['pH', 'alcohol']], y_4_7.tolist(), 30000, 0.001)

.....................................................

KeyboardInterrupt: 

In [19]:
interactive(plot_performance, performance=fixed(adaline_logs), wine_data=fixed(data_4_7), good_threshold=(0, 10),\
            bad_threshold=(0, 10), epoch=(0, len(adaline_logs) - 1), debug=False)

interactive(children=(IntSlider(value=5, description='good_threshold', max=10), IntSlider(value=5, description…

In [20]:
normalized_data_4_7 = normalized_red_wine[(red_wine['quality'] >= 7) | (red_wine['quality'] <= 4)].reset_index(drop=True)

In [66]:
model = adaline(2)
adaline_logs_normalized = model.train(normalized_data_4_7[['pH', 'alcohol']], y_4_7.tolist(), 30000, 0.1)

..................................................................................[8269, 329.962429384063, [-4.00187175480784, 4.712259290917971], 1.1024108563495156] 329.962429384063

DONE
cycle = 8269, errors = 329.962429384063
final weights =  [-4.00187175480784, 4.712259290917971] , bias =  1.1024108563495156


In [69]:
pred = [model.predict(normalized_data_4_7.loc[i, ['pH', 'alcohol']]) for i in range(len(normalized_data_4_7))]
y = [1 if i == True else 0 for i in normalized_data_4_7['quality'] >= 7]

print(s)
print(sum([(y[i] - pred[i]) for i in range(len(y))]), '/', len(normalized_data_4_7))

0.5321428571428571
68 / 280


In [23]:
interactive(plot_performance, performance=fixed(adaline_logs_normalized), wine_data=fixed(normalized_data_4_7), good_threshold=(0, 10),\
            bad_threshold=(0, 10), epoch=(0, len(adaline_logs_normalized) - 1), debug=False)

interactive(children=(IntSlider(value=5, description='good_threshold', max=10), IntSlider(value=5, description…

# V.4

# a)

In [24]:
def holdout(input_data, percentage=30):
    data = input_data.sample(frac=1).reset_index(drop=True)
    sep = int(len(data) * percentage / 100)
    return (data[:sep], data[sep:])

In [25]:
train, test = holdout(normalized_data_4_7)
print(len(normalized_data_4_7), len(train), len(test))

280 84 196


# b)

In [26]:
def k_folds(input_data, k=10, shuffle=False):
    data = input_data.sample(frac=1).reset_index(drop=True) if shuffle else input_data
    seps = [int(len(data) / k) * i for i in range(k + 1)]
    parts = []
    print(len(seps))
    for i in range(k):
        parts.append(data[seps[i]:seps[i + 1]])
    ret = []
    for i in range(k):
        train = pd.concat([parts[j] for j in range(k) if not j == i])
        test = parts[i]
        ret.append((train, test))
    return ret

In [27]:
data = k_folds(normalized_data_4_7)
for train, test in data:
    print(len(normalized_data_4_7), len(train), len(test))

11
280 252 28
280 252 28
280 252 28
280 252 28
280 252 28
280 252 28
280 252 28
280 252 28
280 252 28
280 252 28


# c)

In [28]:
def cross_validate(input_data, y, k=10, epochs=10000, alpha=10):
    data = k_folds(input_data)
    error = []
    logs = []
    for train, test in data:
        model = adaline(2)
        log = model.train(train[['pH', 'alcohol']], train['quality'].tolist(), epochs, alpha)
        logs.append(log)
        y = [1 if i == True else 0 for i in train['quality'] >= 7]
        pred = [model.predict(train.loc[i, ['pH', 'alcohol']]) for i in range(len(train))]
        error.append(sum([abs(y[i] - pred[i]) for i in range(len(y))]))
    return (sum(error) / k)

# V.5 Adventures in the Nth dimension

## a)

In [37]:
model = perceptron(11)
adaline_logs = model.train(normalized_data_4_7[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']], y_4_7, 30000, 0.01)

Weights[0] = 0.6394267984578837
Weights[1] = 0.025010755222666936
Weights[2] = 0.27502931836911926
Weights[3] = 0.22321073814882275
Weights[4] = 0.7364712141640124
Weights[5] = 0.6766994874229113
Weights[6] = 0.8921795677048454
Weights[7] = 0.08693883262941615
Weights[8] = 0.4219218196852704
Weights[9] = 0.029797219438070344
Weights[10] = 0.21863797480360336
bias =  0.5053552881033624 

............................................................

KeyboardInterrupt: 

It converges when data is separable in that hyperplane

## b)

It's called a hyperplane. For 2-dimensional space it's a point, for 3-d it's line and so on.

V.6 Marvin's rebuttal

In [44]:
pan_gal = pd.read_csv("./resources/resources/winequality-red.csv", sep=';')

In [ ]:
def process_to_poly(data):
    poly = data.values.tolist()
    length = len(tmp[0])
    for line in pan_gal_poly:
        for i in range(length):
            for j in range(length):
                line.append(line[i] * line[j])
        for i in range(length):
            for j in range(length):
                for k in range(length):
                    line.append(line[i] * line[j] * line[k])
    print('Was:', length, 'Now:', len(pan_gal_poly[0]))
    return poly

In [51]:
pan_gal_poly = pan_gal.values.tolist()
length = len(tmp[0])
print(length)
for line in pan_gal_poly:
    for i in range(length):
        for j in range(length):
            line.append(line[i] * line[j])
    for i in range(length):
        for j in range(length):
            for k in range(length):
                line.append(line[i] * line[j] * line[k])
print(len(pan_gal_poly[0]))

12
1884


In [38]:
%load_ext Cython

In [43]:
%%cython
import pandas as pd
from libc.stdlib cimport malloc, free, rand, srand, RAND_MAX
from cpython cimport array
import array

red_wine = pd.read_csv("./resources/resources/winequality-red.csv", sep=';')
data = red_wine[(red_wine['quality'] >= 7) | (red_wine['quality'] <= 4)].reset_index()
y = [1 if i == True else 0 for i in data['quality'] >= 8]
data = data[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']].values.tolist()

def process_to_poly(data):
    poly = data.values.tolist()
    length = len(tmp[0])
    for line in pan_gal_poly:
        for i in range(length):
            for j in range(length):
                line.append(line[i] * line[j])
        for i in range(length):
            for j in range(length):
                for k in range(length):
                    line.append(line[i] * line[j] * line[k])
    print('Was:', length, 'Now:', len(pan_gal_poly[0]))
    return poly
data = 
def display(ret, dot=100, detail=10000, final=False):
    """Displays text info about training process"""
    if final:
        print()
        print('DONE')
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
        print('final weights = ', ret[-1][2], ', bias = ', ret[-1][3])
    if ret[-1][0] == 0:
        return
    if ret[-1][0] % dot == 0:
        print('.', end='')
    if ret[-1][0] % detail == 0:
        print()
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
        print('weights change = ', ret[-1][2][0] - ret[-detail - 1][2][0],
              ret[-1][2][1] - ret[-detail - 1][2][1])
        print('weights = ', ret[-1][2], 'bias = ', ret[-1][3])
        print('bias change = ', ret[-1][3] - ret[-detail - 1][3])
        
cdef struct s_perceptron:
    double *weights;
    double bias;
    int amount;
ctypedef s_perceptron t_perceptron

cdef t_perceptron init_perceptron(int amount):
    srand(42)
    cdef t_perceptron ret
    ret.amount = amount
    ret.weights = <double *>malloc(sizeof(double) * amount)
    ret.bias = <double>rand() / RAND_MAX
#     ret.bias = 0.27502931836911926
#     ret.weights[0] = 0.6394267984578837
#     ret.weights[1] = 0.025010755222666936
    for i in range(amount):
        ret.weights[i] = <double>rand() / RAND_MAX
        print('Weights[', i, '] = ', ret.weights[i], sep='')
    print('bias = ', ret.bias, '\n')
    return ret
    
cdef int activation(double value):
    return 1 if value > 0 else 0
    
cdef double predict(t_perceptron *model, list data, double alpha, int y):
    cdef double rez = model.bias
    for i in range(model.amount):
        rez += data[i] * model.weights[i]
    tmp = rez
    rez = activation(rez)
    if y == -1:
        return activation(rez)
    if rez != y:
        for i in range(model.amount):
            model.weights[i] += alpha * data[i] * (y - rez)
        model.bias += alpha * (y - rez)
    return rez != y

cdef list train(t_perceptron model, list data, list y, int epochs, double alpha):
    cdef list ret = []
    cdef double errors = 1
    cdef unsigned long i = 0
    if epochs == 0:
        while (errors):
            errors = 0
            for j, item in enumerate(data):
                errors += predict(&model, item, alpha, y[j])
            ret.append([i, errors, [model.weights[i] for i in range(model.amount)],
                        model.bias])
            i += 1
            display(ret)
        display(ret, final=True)
    else:
        for i in range(epochs):
            errors = 0
            for j, item in enumerate(data):
                errors += predict(&model, item, alpha, y[j])
            ret.append([i, errors, [model.weights[i] for i in range(model.amount)],
                        model.bias])
            display(ret)
            if errors == 0:
                break
        display(ret, final=True)
    return ret

cdef t_perceptron model = init_perceptron(11)
train(model, data, y, 10000000, 0.0000001)
free(model.weights)

Weights[0] = 0.5245871020129822
Weights[1] = 0.7354235321913956
Weights[2] = 0.26330554078487006
Weights[3] = 0.37622397131110724
Weights[4] = 0.19628582577979464
Weights[5] = 0.9758738810084173
Weights[6] = 0.512318108469396
Weights[7] = 0.5304490451377114
Weights[8] = 0.2571016295147602
Weights[9] = 0.10708725457409735
Weights[10] = 0.8154876268540917
bias =  0.00032870750889587566 

....................................................................................................
cycle = 10000, errors = 52.0
weights change =  -0.5166780499996315 -0.030206160497907253
weights =  [0.0076829020133507295, 0.7052047711934891, 0.24497983478481788, 0.205816976311079, 0.1910769797798061, 0.5048138810071049, -0.5172696915334013, 0.469923194115813, 0.05521953151586182, 0.06848433557434076, 0.13773986185549209] bias =  -0.06043809249181213
bias change =  -0.06074060000070801
....................................................................................................
cycle = 20000, er

....................................................................................................
cycle = 170000, errors = 39.0
weights change =  -0.0001911400001075418 -0.012080730498363224
weights =  [-0.028049267985586195, 0.5362359762303287, 0.31229471977953144, -0.004972093688023844, 0.17413418418028015, 0.0026872810108627154, -0.0014919915374832502, 0.4179699836298492, -0.16267707248494312, 0.08478198757270573, -0.005597318143986204] bias =  -0.11236319249330527
bias change =  -0.0026315000000756705
....................................................................................................
cycle = 180000, errors = 39.0
weights change =  0.0001757299999112054 -0.011562584998146996
weights =  [-0.02787353798567499, 0.5246733912321817, 0.3154959147793092, -0.0075607136879988854, 0.17266242758052583, 0.0016234810108658234, -0.0013255915374809839, 0.41558026383270014, -0.17286467148520998, 0.08583626657261921, -0.00033686814398074447] bias =  -0.11474339249337372
bias chan

....................................................................................................
cycle = 330000, errors = 30.0
weights change =  0.0007593499999386626 -0.006722253001605005
weights =  [-0.030741157986583566, 0.3916410817112809, 0.33603644077840156, -0.02572124368901778, 0.15534827088545966, 0.002914981010878705, -0.0009831915374615794, 0.39604703743779235, -0.2579115184878203, 0.09585060956884439, 0.0376399318556422] bias =  -0.13413359249393128
bias change =  -0.0004726000000135899
....................................................................................................
cycle = 340000, errors = 30.0
weights change =  -0.00034163000005702285 -0.005875533001448052
weights =  [-0.03108278798664059, 0.3857655487098329, 0.3362131307784304, -0.0263344136891179, 0.15431962048591233, 0.002927681010879337, -0.0010279915374598514, 0.39581096836212776, -0.2590789884862303, 0.09594835956855972, 0.038888851855522856] bias =  -0.13435929249393777
bias change =  -0.000

....................................................................................................
cycle = 490000, errors = 30.0
weights change =  0.00013998999989255773 -0.002981339000683203
weights =  [-0.03302022798757684, 0.3206763241947853, 0.3283948437875776, -0.017910273690167105, 0.13913602958910154, 0.0025827810108755653, -0.0008962915374408112, 0.39478022353932746, -0.26135084446349216, 0.09279140556399054, 0.043556261853322745] bias =  -0.13529029249396454
bias change =  0.00010290000000295896
....................................................................................................
cycle = 500000, errors = 29.0
weights change =  0.00010741999989331147 -0.0030011425006825077
weights =  [-0.03291280798768353, 0.3176751816941028, 0.32724735178867154, -0.017468223690211452, 0.1381540662891219, 0.0025314810108778454, -0.0008735915374385486, 0.3948785391644604, -0.2606423644620294, 0.09221464856368135, 0.043395771853114475] bias =  -0.1351874924939616
bias change =  0

....................................................................................................
cycle = 650000, errors = 29.0
weights change =  -6.532000011941208e-05 -0.0023672060003536166
weights =  [-0.03245352798935582, 0.27607231918564584, 0.3087958258073887, -0.01430802869029294, 0.12399056658942348, 0.0021304810109183917, -0.0007113915374059756, 0.39727779801421353, -0.24622478444482646, 0.08301698155895731, 0.04119588184990932] bias =  -0.1327122924938904
bias change =  0.0002689000000077324
....................................................................................................
cycle = 660000, errors = 28.0
weights change =  -6.759000011941768e-05 -0.002366954000353416
weights =  [-0.03252111798947524, 0.2737053651852924, 0.30743978180888193, -0.014174068690278962, 0.12311242488942081, 0.002129581010921765, -0.000712491537403939, 0.39754078095632966, -0.244832626444803, 0.08233462555863723, 0.04099803184968275] bias =  -0.13244349249388268
bias change =  0.000

...

....................................................................................................
cycle = 1290000, errors = 29.0
weights change =  0.00017221999997759155 -0.0015550319994759587
weights =  [-0.023953297992625983, 0.13773691920553516, 0.21078268586846402, -0.01090847368960086, 0.06300924138516278, 0.0006553810109397059, -0.0002646915373609037, 0.40806442655149067, -0.1723217074441203, 0.03382365755281593, 0.025978271841852328] bias =  -0.12156009249356972
bias change =  0.00015510000000446
....................................................................................................
cycle = 1300000, errors = 28.0
weights change =  0.00017183999997755564 -0.0015538169994760986
weights =  [-0.023781457992648428, 0.13618310220605906, 0.20891552386872597, -0.010916723689598121, 0.0621081432851543, 0.0006368810109387081, -0.0002689915373612542, 0.4082125239946382, -0.17109133844402347, 0.033033324552876414, 0.02570563184187525] bias =  -0.12140509249356526
bias change

....................................................................................................
cycle = 1450000, errors = 30.0
weights change =  0.00018247999998343908 -0.0013590560001427027
weights =  [-0.021145007992961422, 0.11366823720851697, 0.18072352987241808, -0.010434103689562792, 0.04855467998664913, 0.0004245810109246784, -0.00022399153736754067, 0.4103136315650821, -0.15255857844251686, 0.021278283552369943, 0.02136225184222339] bias =  -0.11920329249350195
bias change =  0.00012410000000356858
....................................................................................................
cycle = 1460000, errors = 28.0
weights change =  0.00017665999998343201 -0.0013591700001426843
weights =  [-0.02096834799297799, 0.11230906720837429, 0.17881027987262138, -0.010295703689561472, 0.047643900086743254, 0.00040948101092378304, -0.00021849153736810167, 0.4104309335762873, -0.15131197644240688, 0.02051196755232148, 0.021021781842247397] bias =  -0.1190797924934984
bias

....................................................................................................
cycle = 1610000, errors = 27.0
weights change =  0.00025895999997015487 -0.0013714800001501104
weights =  [-0.017536627993291556, 0.09174767520617938, 0.15027967187575944, -0.008482503689539778, 0.03346115088810176, 0.0001245810109142569, -0.00011169153737706067, 0.41181621343712344, -0.1335660764411766, 0.008785645551869234, 0.015959151842629273] bias =  -0.11761029249345614
bias change =  7.16000000020589e-05
....................................................................................................
cycle = 1620000, errors = 30.0
weights change =  0.00016323999996986638 -0.0013162000001518126
weights =  [-0.01737338799332169, 0.09043147520602757, 0.1483893698759752, -0.008303063689539515, 0.03247612848819013, 0.00012198101091389987, -0.00011009153737668657, 0.4118878940172431, -0.1323739144411544, 0.008040095551852004, 0.015640231842659297] bias =  -0.11753409249345395
bias c

....................................................................................................
cycle = 1770000, errors = 28.0
weights change =  0.00015168000004529142 -0.0011194370002081955
weights =  [-0.014986627993542005, 0.07189386570299992, 0.11967599187846789, -0.00603562368960311, 0.018122438688862267, 0.00013218101091205548, -7.639153737098726e-05, 0.4121291983664832, -0.11590697344229996, -0.0005594654481140481, 0.011358681842701198] bias =  -0.11724639249344568
bias change =  -5.4000000001552806e-05
....................................................................................................
cycle = 1780000, errors = 31.0
weights change =  0.00015204000004650012 -0.0011175345001891573
weights =  [-0.014834587993495504, 0.07077633120281077, 0.11773210787854999, -0.0059436936896079175, 0.01720510078890886, 0.00012993101091170444, -8.12915373706878e-05, 0.4120716756826397, -0.11493212344247189, -0.0006725704481124335, 0.01110601184271609] bias =  -0.1173033924934473

KeyboardInterrupt: 